<a href="https://colab.research.google.com/github/A-Peoples/BCS-Recreated/blob/main/BCS_Recreated_1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# installing sportsdataverse
!pip install sportsdataverse
!pip install xgboost==1.6.2

  Using cached sportsdataverse-0.0.39-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.6/19.6 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.3/418.3 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.25.2
    Uninstalling polars-1.25.2:
      Successfully uninstalled polars-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-polars-cu12 25.6.0 requires polars<1.29,>=1.25, but you have polars 0.18.15 which is incompatible.


In [5]:
import sportsdataverse as sdv
import pandas as pd
import numpy as np

In [6]:
data = sdv.cfb.load_cfb_schedule([2025], return_as_pandas=True)
data["margin"] = data["home_points"] - data["away_points"]
team_names = data['home_team'].unique()

data = data.dropna(subset=['home_points', 'away_points'])

data.shape

100%|██████████| 1/1 [00:00<00:00,  2.08it/s]


(2517, 32)

In [7]:
data["margin"] = data["home_points"] - data["away_points"]

final_team = pd.DataFrame()
wins = 0
losses = 0
c_wins = 0
c_losses = 0
margin = 0
total_games = 0
conference = ''
for team_name in team_names:
  points_scored = 0
  points_against = 0
  eilo = 0
  data["margin"] = data["home_points"] - data["away_points"]
  data
  team = data.loc[(data['home_team'] == team_name) | (data['away_team'] == team_name)]
  team
  main_team = pd.DataFrame()
  for index, row in team.iterrows():
    if row["home_team"] == team_name:
      points_scored = points_scored + row['home_points']
      points_against = points_against + row['away_points']
      total_games +=1
      eilo = eilo + row["home_pregame_elo"]
      if row["home_points"] >  row["away_points"]:
        if row["conference_game"] == True:
          c_wins +=1
        wins +=1
      elif row["home_points"] <  row["away_points"]:
        if row["conference_game"] == True:
          c_losses +=1
        losses +=1
      margin += row["margin"]
      conference = row['home_conference']
    elif row["away_team"] == team_name:
      points_against = points_against + row['home_points']
      points_scored = points_scored + row['away_points']
      eilo = eilo + row["away_pregame_elo"]
      total_games +=1
      if row["home_points"] <  row["away_points"]:
        if row["conference_game"] == True:
          c_wins +=1
        wins +=1
      elif row["home_points"] >  row["away_points"]:
        if row["conference_game"] == True:
          c_losses +=1
        losses +=1
      margin += row["margin"]
      conference = row['away_conference']
      points_scored = points_scored / total_games
      points_against = points_against / total_games
  main_team = pd.DataFrame({
      "team": [team_name],
      "wins": [wins],
      "losses": [losses],
      "c_wins": [c_wins * 0.5],
      "c_losses": [c_losses * 1.5],
      'conference': [conference],
      'points_scored': [points_scored],
      'points_against': [points_against],
      "margin": [margin],
      'eilo': [eilo],
      'games_played': [total_games]
      })
  final_team = pd.concat([final_team, main_team], ignore_index=True)
  wins = 0
  losses = 0
  c_wins = 0
  c_losses = 0
  margin = 0
  total_games = 0

In [8]:
FBS = ('SEC', 'Sun Belt', 'Big Ten', 'Pac-12', 'Conference USA', 'FBS Independents', 'ACC', 'Mountain West', 'Big 12', "Mid American")

In [9]:
FBS = final_team.loc[final_team['conference'].isin(['SEC', 'Sun Belt', 'Big Ten', 'Pac-12', 'Conference USA', 'FBS Independents', 'ACC', 'Mountain West', 'Big 12', "Mid American"])]

FBS.eilo = FBS['eilo'] / FBS['games_played']

FBS["points"] = ((FBS['wins'] * 15) - (FBS['losses']) * 15) + ((FBS['c_wins'] * 15) - (FBS['c_losses']) * 15) + (FBS['eilo'] / 25)

FBS = FBS.sort_values(by=["points"], ascending=False)
FBS["rank"] = range(1, len(FBS) + 1)
FBS = FBS.head(25)
FBS = FBS[["rank","team","conference", "wins", "losses", "points", 'margin','eilo']]
FBS

/tmp/ipython-input-2132751125.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FBS.eilo = FBS['eilo'] / FBS['games_played']
/tmp/ipython-input-2132751125.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FBS["points"] = ((FBS['wins'] * 15) - (FBS['losses']) * 15) + ((FBS['c_wins'] * 15) - (FBS['c_losses']) * 15) + (FBS['eilo'] / 25)


,rank,team,conference,wins,losses,points,margin,eilo
105,1,Indiana,Big Ten,8,0,234.61,246,1927.75
149,2,Texas A&M,SEC,8,0,227.455,57,1748.875
174,3,BYU,Big 12,8,0,224.76,66,1681.5
328,4,Georgia Tech,ACC,8,0,220.26,94,1569.0
86,5,Ohio State,Big Ten,7,0,215.64,74,2016.0
405,6,Alabama,SEC,7,1,204.185,131,1917.125
56,7,Cincinnati,Big 12,7,1,188.335,84,1520.875
127,8,Virginia,ACC,7,1,181.415,118,1535.375
118,9,Oregon,Big Ten,7,1,176.43,78,1973.25
166,10,Ole Miss,SEC,7,1,175.04,98,1938.5
